In [15]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [16]:
# def function_name():
def extract_transform_load(wiki_file,kaggle_file,ratings_file):
    
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    ratings = pd.read_csv(ratings_file, low_memory=False)
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    
    # 3. Open the read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as file:
        json_file = json.load(file)
        
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    wiki_movies_df = pd.DataFrame(json_file)
          
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings

In [17]:
# 6 Create the path to your file directory and variables for the three files. 
file_dir = "C:/UC_Berkeley/Analysis_Projects_Class_Folder/Module8_ETL/Data_for_Movies_ETL"

# Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

# file_paths = [wiki_file,kaggle_file,ratings_file]

In [18]:
# 7. Set the three variables in Step 6 equal to the function created in Step 1.
#wiki_file, kaggle_file, ratings_file = extract_transform_load(file_paths)
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file,kaggle_file,ratings_file)
# wiki_file_df = read_json(wiki_file)
# kaggle_file_df =  read_csv_data(kaggle_file)
# ratings_file_df = read_csv_data(ratings_file)

In [19]:
len(wiki_file)

7311

In [20]:
# Some records in the middle
wiki_file[3600:3605]

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
3600,https://en.wikipedia.org/wiki/Benji:_Off_the_L...,2004.0,https://www.imdb.com/title/tt0315273/,Benji: Off the Leash!,Joe Camp,NaN,NaN,NaN,NaN,"[Benji, Nick Whitaker, Shaggy, Gypsy the Cocka...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3601,https://en.wikipedia.org/wiki/The_Best_Thief_i...,2004.0,https://www.imdb.com/title/tt0389796/,The Best Thief in the World,Jacob Kornbluth,"[Tim Perrell, Nicola Usborne]",NaN,NaN,NaN,"[Marc Rozendaal, Michael Silverman, David Wars...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3602,https://en.wikipedia.org/wiki/Beyond_the_Sea_(...,2004.0,https://www.imdb.com/title/tt0363473/,Beyond the Sea,Kevin Spacey,"[Kevin Spacey, Phillip Barry, Dana Brunetti, A...",NaN,NaN,NaN,"[Kevin Spacey, Kate Bosworth, Bob Hoskins, Joh...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3603,https://en.wikipedia.org/wiki/The_Big_Bounce_(...,2004.0,https://www.imdb.com/title/tt0315824/,The Big Bounce,George Armitage,"[George Armitage, Steve Bing]",Sebastian Gutierrez,NaN,"[The Big Bounce, by, Elmore Leonard]","[Owen Wilson, Charlie Sheen, Morgan Freeman, S...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3604,https://en.wikipedia.org/wiki/Birth_(film),2004.0,https://www.imdb.com/title/tt0337876/,Birth,Jonathan Glazer,"[Lizie Gower, Nick Morris, Jean-Louis Piel]",NaN,NaN,NaN,"[Nicole Kidman, Cameron Bright, Danny Huston, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# 8. Set the DataFrames from the return statement equal to the file names in Step 6. 
wiki_movies_df = wiki_file
kaggle_metadata = kaggle_file
ratings = ratings_file

In [23]:
# 9. Check the wiki_movies_df DataFrame.
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 10. Check the kaggle_metadata DataFrame.
kaggle_metadata.head()

In [ ]:
# 11. Check the ratings DataFrame.
ratings.head()

In [4]:
wiki_file_path= f'{file_dir}/wikipedia-movies.json'
wiki_movies_raw = None
with open(wiki_file_path, mode='r') as file:
    wiki_movies_raw = json.load(file)

In [ ]:
for x in wiki_movies_raw[0]:
    print(x,wiki_movies_raw[0][x])
    


In [ ]:
for x in wiki_movies_raw[0].values():
    print(x)

In [ ]:
len(wiki_movies_raw)

In [ ]:
#wiki_movies_raw_movies

In [77]:
wiki_movies_raw_movies = []

for movie in wiki_movies_raw:
    print(movie["year"]
#     if movie["year"]>=1990:
#         wiki_movies_raw_movies.append(movie)
        
# print(wiki_movies_raw_movies)

SyntaxError: unexpected EOF while parsing (<ipython-input-77-dc3ee6297def>, line 8)

In [10]:
wiki_movies_raw_movies = []
for movie in wiki_movies_raw:
    if "year" in movie:
        if movie["year"]>=1990:
            wiki_movies_raw_movies.append(movie)
        
print(wiki_movies_raw_movies[0:10])

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane', 'year': 1990, 'imdb_link': 'https://www.imdb.com/title/tt0098987/', 'title': 'The Adventures of Ford Fairlane', 'Directed by': 'Renny Harlin', 'Produced by': ['Steve Perry', 'Joel Silver'], 'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'], 'Story by': ['David Arnott', 'James Cappe'], 'Based on': ['Characters', 'by Rex Weiner'], 'Starring': ['Andrew Dice Clay', 'Wayne Newton', 'Priscilla Presley', 'Lauren Holly', 'Morris Day', 'Robert Englund', "Ed O'Neill"], 'Narrated by': 'Andrew "Dice" Clay', 'Music by': ['Cliff Eidelman', 'Yello'], 'Cinematography': 'Oliver Wood', 'Edited by': 'Michael Tronick', 'Productioncompany ': 'Silver Pictures', 'Distributed by': '20th Century Fox', 'Release date': ['July 11, 1990', '(', '1990-07-11', ')'], 'Running time': '102 minutes', 'Country': 'United States', 'Language': 'English', 'Budget': '$20 million', 'Box office': '$21.4 million'}, {'url': 'https://en.wikipe

In [ ]:
dir([])

In [60]:
a_dict = {'one': 1, 'two': 2, 'thee': 3, 'four': 4}

In [13]:
wiki_movies_raw_movies_test = []
for movie in wiki_movies_raw:
    if ("Director" in movie or "Directed by" in movie) and ("imdb_link" in movie) and ("No of episodes" not in movie):
        wiki_movies_raw_movies_test.append(movie)
        
print(wiki_movies_raw_movies_test[0:10])

[{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane', 'year': 1990, 'imdb_link': 'https://www.imdb.com/title/tt0098987/', 'title': 'The Adventures of Ford Fairlane', 'Directed by': 'Renny Harlin', 'Produced by': ['Steve Perry', 'Joel Silver'], 'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'], 'Story by': ['David Arnott', 'James Cappe'], 'Based on': ['Characters', 'by Rex Weiner'], 'Starring': ['Andrew Dice Clay', 'Wayne Newton', 'Priscilla Presley', 'Lauren Holly', 'Morris Day', 'Robert Englund', "Ed O'Neill"], 'Narrated by': 'Andrew "Dice" Clay', 'Music by': ['Cliff Eidelman', 'Yello'], 'Cinematography': 'Oliver Wood', 'Edited by': 'Michael Tronick', 'Productioncompany ': 'Silver Pictures', 'Distributed by': '20th Century Fox', 'Release date': ['July 11, 1990', '(', '1990-07-11', ')'], 'Running time': '102 minutes', 'Country': 'United States', 'Language': 'English', 'Budget': '$20 million', 'Box office': '$21.4 million'}, {'url': 'https://en.wikipe

In [24]:
wiki_movies_box_office = wiki_movies_df["Box office"].dropna()
wiki_movies_box_office

0          $21.4 million
1           $2.7 million
2            $57,718,089
3             $7,331,647
4       $6,939,946 (USA)
              ...       
7305       $19.4 million
7306       $41.9 million
7307       $76.1 million
7308       $38.4 million
7309        $5.5 million
Name: Box office, Length: 5530, dtype: object